# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: Chuyu Wu
Date: 8/16


## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [8]:
import os, pathlib, datetime as dt
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [9]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [10]:
SYMBOL = 'AAPL'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    print(js)
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date'})[['date']]
    df_api['date'] = pd.to_datetime(df_api['date'])

v_api = validate(df_api, ['date']); v_api

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'AAPL', '3. Last Refreshed': '2025-08-15', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2025-08-15': {'1. open': '234.0000', '2. high': '234.2800', '3. low': '229.3350', '4. close': '231.5900', '5. volume': '56038657'}, '2025-08-14': {'1. open': '234.0550', '2. high': '235.1200', '3. low': '230.8500', '4. close': '232.7800', '5. volume': '51916275'}, '2025-08-13': {'1. open': '231.0700', '2. high': '235.0000', '3. low': '230.4300', '4. close': '233.3300', '5. volume': '69878546'}, '2025-08-12': {'1. open': '228.0050', '2. high': '230.8000', '3. low': '227.0700', '4. close': '229.6500', '5. volume': '55672301'}, '2025-08-11': {'1. open': '227.9200', '2. high': '229.5600', '3. low': '224.7600', '4. close': '227.1800', '5. volume': '61806132'}, '2025-08-08': {'1. open': '220.8300', '2. high': '231.0000', '3. low': '219.2500', '4. close': '229.3500', '5

{'missing': [], 'shape': (100, 1), 'na_total': 0}

In [11]:
%pip install yfinance
# Install yfinance for fallback API data pull if needed

Note: you may need to restart the kernel to use updated packages.


In [12]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved data\raw\api_source-alpha_symbol-AAPL_20250818-211552.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [ ]:
SCRAPE_URL = 'https://sg.finance.yahoo.com/markets/stocks/gainers/'  
headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

{'missing': [], 'shape': (25, 12), 'na_total': 25}

In [14]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='markets')

Saved data\raw\scrape_site-example_table-markets_20250818-211556.csv


## Documentation
- API Source: (URL/endpoint/params)
- Scrape Source: (URL/table description)
- Assumptions & risks: (rate limits, selector fragility, schema changes)
- Confirm `.env` is not committed.